# NLP with CNN

### Exercise objectives:

- Use CNN instead of RNN for NLP

<hr>
<hr>


# The data


❓ **Question** ❓ Let's first load the data. You don't have to understand what is going on in the function, it does not matter here.

⚠️ **Warning** ⚠️ The `load_data` function has a `percentage_of_sentences` argument. Depending on your computer, there are chances that too many sentences will make your compute slow down, or even freeze - your RAM can overflow. For that reason, **you should start with 10% of the sentences** and see if your computer handles it. Otherwise, rerun with a lower number. 

⚠️ **DISCLAIMER** ⚠️ **No need to play _who has the biggest_ (RAM) !** The idea is to get to run your models quickly to prototype. Even in real life, it is recommended that you start with a subset of your data to loop and debug quickly. So increase the number only if you are into getting the best accuracy.

In [2]:
######################################
### Run this cell to load the data ###
######################################

import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import text_to_word_sequence

def load_data(percentage_of_sentences=None):
    train_data, test_data = tfds.load(name="imdb_reviews", split=["train", "test"], batch_size=-1, as_supervised=True)

    train_sentences, y_train = tfds.as_numpy(train_data)
    test_sentences, y_test = tfds.as_numpy(test_data)
    
    # Take only a given percentage of the entire data
    if percentage_of_sentences is not None:
        assert(percentage_of_sentences> 0 and percentage_of_sentences<=100)
        
        len_train = int(percentage_of_sentences/100*len(train_sentences))
        train_sentences, y_train = train_sentences[:len_train], y_train[:len_train]
  
        len_test = int(percentage_of_sentences/100*len(test_sentences))
        test_sentences, y_test = test_sentences[:len_test], y_test[:len_test]
    
    X_train = [text_to_word_sequence(_.decode("utf-8")) for _ in train_sentences]
    X_test = [text_to_word_sequence(_.decode("utf-8")) for _ in test_sentences]
    
    return X_train, y_train, X_test, y_test

X_train, y_train, X_test, y_test = load_data(percentage_of_sentences=10)

In [11]:
y_train

array([0, 0, 0, ..., 1, 0, 0])

Remember that to do NLP, you need to go through one of the following options, as shown here: 

<img src="embedding_or_RNN.png" width="700px" />

But in both cases, you can replace the recurrent layer (top part) by a CNN layer. We will go into both, starting from the one on the left were the embedding is learned in the network.




# Part 1 : Concatenate a Keras Embedding with a Conv1D🔥 

Let's train a fancy network here. 

Each of your words is represented by a vector of size N (the size of your embedding). Therefore, as a sentence is a sequence of words, it is represented by a matrix (number of words, N). So, all your sentences are actually represented as matrices once embedded.

If you think about it, an image is also a matrix. Said differently, you may represent your sentence of word as a matrix, where each column (or row, depending on how you want to look at it) is a word, and each row (or each column) corresponds to a coordinate in the embedding space. As shown here

<img src="image_comparison.png" width="500px" />

Well, in that case, as these are close to images, why not using convolution on them? Yes, convolutions!
But, be careful. In the case of images, convolutions are 2 dimensional as the filters can move up and down, and left and right. In the case of our sentences, we want the kernel to move _only_ in the word by word direction (The alternative, moving coordinate by coordinate of the embedding space doesn't make much sense).

So let's create a model that use convolutions.

## First, the data

❓ **Question** ❓ You will need to prepare your data. First, tokenize them. Then, you need to pad them (use a value `maxlen` equal to 150). You also might need to compute the size of your vocabulary ;)

In [4]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()

tokenizer.fit_on_texts(X_train)

X_train_token = tokenizer.texts_to_sequences(X_train)
X_test_token = tokenizer.texts_to_sequences(X_test)

In [5]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [7]:
X_train_pad = pad_sequences(X_train_token,maxlen=150,padding='post', dtype='float32',value=0.0)
X_test_pad = pad_sequences(X_test_token,maxlen=150,padding='post', dtype='float32',value=0.0)

In [8]:
vocab_size = len(tokenizer.word_index) +1
vocab_size

30420

## Using 1D Convolution.

❓ **Question** ❓ Define a model that has :
- an `Embedding` layer: `input_dim` is the `vocab_size + 1`, `output_dim` is the embedding space dimension, and `mask_zero` has to be set to `True`. Here, for computational reasons, set `input_length` to the maximum length of your observations (that you just defined in the previous question).
- a `Conv1D` layer 
- a `Flatten` layer
- a `Dense` layer
- an output layer

Compile the model accordingly

❗ **Remark** ❗ The size of the `Conv1D` kernel corresponds exactly to the number of side-by-side words (tokens) each kernel is taking into account ;)

In [13]:
from tensorflow.keras import layers, regularizers, Sequential
from tensorflow.keras.callbacks import EarlyStopping

model = Sequential()

model.add(layers.Embedding(input_dim=vocab_size, input_length=150, output_dim=100,mask_zero=True))
model.add(layers.Conv1D(20,kernel_size=3))
model.add(layers.Flatten())
model.add(layers.Dense(40,activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 150, 100)          3042000   
                                                                 
 conv1d_1 (Conv1D)           (None, 148, 20)           6020      
                                                                 
 flatten_1 (Flatten)         (None, 2960)              0         
                                                                 
 dense_1 (Dense)             (None, 40)                118440    
                                                                 
 dense_2 (Dense)             (None, 1)                 41        
                                                                 
Total params: 3,166,501
Trainable params: 3,166,501
Non-trainable params: 0
_________________________________________________________________


❓ **Question** ❓ Look at the number of parameters. You can compare it to the model that you had in previous exercise (esp. the first one)

In [15]:
es = EarlyStopping(patience=4, restore_best_weights=True)

history = model.fit(X_train_pad,
                   y_train,
                   validation_split=0.3,
                   batch_size=64,
                   epochs=20,
                   callbacks=[es])

model.evaluate(X_test_pad, y_test)

Epoch 1/20
28/28 [==============================] - 1s 40ms/step - loss: 0.0277 - accuracy: 0.9966 - val_loss: 0.5301 - val_accuracy: 0.7507
Epoch 2/20
28/28 [==============================] - 1s 41ms/step - loss: 0.0041 - accuracy: 0.9989 - val_loss: 0.5323 - val_accuracy: 0.7800
Epoch 3/20
28/28 [==============================] - 1s 38ms/step - loss: 6.4712e-04 - accuracy: 1.0000 - val_loss: 0.5328 - val_accuracy: 0.7853
Epoch 4/20
28/28 [==============================] - 1s 43ms/step - loss: 1.7532e-04 - accuracy: 1.0000 - val_loss: 0.5590 - val_accuracy: 0.7747
Epoch 5/20
79/79 [==============================] - 0s 6ms/step - loss: 0.5591 - accuracy: 0.7372


[0.5590500235557556, 0.7372000217437744]

❓ **Question** ❓ Fit your model with a stopping criterion, and evaluate it on the test data.

You will probably notice that it is ... **much faster** than RNN.

In [17]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(patience=5, restore_best_weights=True)

model.fit(X_train_pad, y_train, 
          epochs=20, 
          batch_size=32,
          validation_split=0.3,
          callbacks=[es]
         )


res = model.evaluate(X_test_pad, y_test, verbose=0)

print(f'The accuracy evaluated on the test set is of {res[1]*100:.3f}%')

Epoch 1/20
55/55 [==============================] - 2s 35ms/step - loss: 0.0027 - accuracy: 0.9994 - val_loss: 0.7877 - val_accuracy: 0.7013
Epoch 2/20
55/55 [==============================] - 2s 31ms/step - loss: 1.5309e-04 - accuracy: 1.0000 - val_loss: 0.6732 - val_accuracy: 0.7427
Epoch 3/20
55/55 [==============================] - 2s 31ms/step - loss: 1.1951e-05 - accuracy: 1.0000 - val_loss: 0.7173 - val_accuracy: 0.7547
Epoch 4/20
55/55 [==============================] - 2s 31ms/step - loss: 2.9179e-06 - accuracy: 1.0000 - val_loss: 0.8265 - val_accuracy: 0.7480
Epoch 5/20
55/55 [==============================] - 2s 30ms/step - loss: 2.2090e-07 - accuracy: 1.0000 - val_loss: 0.8611 - val_accuracy: 0.7507
Epoch 6/20
55/55 [==============================] - 2s 31ms/step - loss: 5.5893e-08 - accuracy: 1.0000 - val_loss: 0.9097 - val_accuracy: 0.7507
Epoch 7/20
55/55 [==============================] - 2s 32ms/step - loss: 2.0763e-08 - accuracy: 1.0000 - val_loss: 0.9394 - val_accura

# Part 2 : Learn a Word2Vec representation, and then feed it into a NN with a `Conv1D`🔥 

In the first part of the exercise, you were asked to jointly learn the embedding representation and the CNN convolution within the same network, which was the CNN counterpart of the left part of this Figure: 

<img src="embedding_or_RNN.png" width="700px" />

Now, let's try to replace the RNN with a CNN for an architecture as on the right side.

❓ **Question** ❓ Learn a word2vec model or load a trained one directly from GENSIM (transfer learning). Then, prepare your data as in the previous exercise. This question is quite long, it prepares you for real world challenges - but you have already done all the building bricks in the previous exercises

In [20]:
from gensim.models import Word2Vec
import numpy as np

word2vec = Word2Vec(X_train)

X_train_emb = [np.array([word2vec.wv[word] for word in sentence if word in word2vec.wv]) for sentence in X_train]
X_test_emb = [np.array([word2vec.wv[word] for word in sentence if word in word2vec.wv]) for sentence in X_test]                        

In [21]:
X_train_pad2 = pad_sequences(X_train_emb,maxlen=150,dtype='float32',value=0.0,padding='post')
X_test_pad2 = pad_sequences(X_test_emb,maxlen=150,dtype='float32',value=0.0,padding='post')

In [59]:
import tensorflow
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras import optimizers

lr_schedule = ExponentialDecay(initial_learning_rate=0.01,decay_steps=1000,decay_rate=0.05)

opt = tensorflow.keras.optimizers.RMSprop(learning_rate=lr_schedule)

In [60]:
model = Sequential()

model.add(layers.Masking(mask_value=0.0))

model.add(layers.Conv1D(40,kernel_size=3))
model.add(layers.Conv1D(40,kernel_size=3))
model.add(layers.MaxPool1D(2))

model.add(layers.Conv1D(40,kernel_size=2))
model.add(layers.Conv1D(40,kernel_size=2))
model.add(layers.MaxPool1D(2))


model.add(layers.Flatten())

model.add(layers.Dense(80,activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(80,activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(40,activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer=opt,loss='binary_crossentropy', metrics=['accuracy'])

❓ **Question** ❓ Now construct a model that has a `Conv1D` layer, a flatten layer, a dense layer, and an output layer. Compile it, and fit it on the train data. You can then evaluate it on the test set.

In [ ]:
es = EarlyStopping(patience=5, restore_best_weights=True)

model.fit(X_train_pad2, y_train, 
          epochs=20, 
          batch_size=32,
          validation_split=0.3,
          callbacks=[es]
         )


res = model.evaluate(X_test_pad2, y_test, verbose=0)
print(f'The accuracy evaluated on the test set is of {res[1]*100:.3f}%')

Epoch 1/20
55/55 [==============================] - 4s 43ms/step - loss: 25.0213 - accuracy: 0.5091 - val_loss: 1.9842 - val_accuracy: 0.4707
Epoch 2/20
55/55 [==============================] - 2s 37ms/step - loss: 0.8375 - accuracy: 0.5166 - val_loss: 0.7035 - val_accuracy: 0.4320
Epoch 3/20
14/55 [======>.......................] - ETA: 0s - loss: 0.8647 - accuracy: 0.4866

❓ **Question** ❓ You might be frustrated by the accuracy you got, this happens to us all at some point. Once you have tested your first iteration, you need to improve your models: by making them more complex, changing the parameters, stacking additional layers, and so on.

Only practice and experimentation will get you there. So you can go back to your previous models, change them and try to get better results ;)